In [1]:
# numpy读取二进制文件

import numpy as np

data_path=r"D:\RFFI-based-on-CVCNN\datasets\ORACLE\neu_m044q5210\KRI-16Devices-RawData\2ft\WiFi_air_X310_3123D7D_2ft_run1.sigmf-data"

f= open(data_path,'rb') 
f.seek(0)
data=f.read()
data=np.frombuffer(data,dtype=np.complex128,count=-1,offset=0)
print(len(data))


20006400


In [8]:
import os
data_path=r"D:\RFFI-based-on-CVCNN\datasets\ORACLE\neu_m044q5210\KRI-16Devices-RawData\2ft"
classes=[]
for file in os.listdir(data_path):
    
    # file name示例:WiFi_air_X310_3123D7B_2ft_run1.sigmf-data
    extension=os.path.splitext(file)[1]
    # print(extension)
    file_class=os.path.splitext(file)[0].split('_')[3]
    
    if extension=='.sigmf-data':
        if file_class not in classes:
            classes.append(file_class)

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np
def TrainDataset(num):
    x = np.load(f"D:/RFFI-based-on-CVCNN/codes/FS-SEI/FS-SEI_4800/Dataset/X_train_{num}Class.npy")
    y = np.load(f"D:/RFFI-based-on-CVCNN/codes/FS-SEI/FS-SEI_4800/Dataset/Y_train_{num}Class.npy")
    x=x[:,:,0]+x[:,:,1]*1j
    x=x.astype(np.complex64)
    x=np.expand_dims(x,axis=(1,3))
    print(type(x[0,0,0,0]))
    y = y.astype(np.uint8)# 8bit数0-255
    X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size = 0.1, random_state= 30)
    return X_train, X_val, Y_train, Y_val

In [20]:
X_train, X_val, Y_train, Y_val=TrainDataset(10)

<class 'numpy.complex64'>


In [4]:
import argparse
import sys
import os
import importlib
import shutil
import time

import torch
import torch.nn as nn
from dataset.RFFIDataset import RFFIDataset
from torch.utils.data import DataLoader,Dataset
from network.STCVCNN import STCVCNN
from utils.strategy import step_lr, accuracy
from utils.plot import draw_curve
from utils.centerloss import CenterLoss
from utils.hard_triplet_loss import HardTripletLoss
from test import predict_unkonwn_sig

from sklearn.model_selection import train_test_split
import numpy as np

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

def TrainDataset(num):
    x = np.load(f"D:/RFFI-based-on-CVCNN/codes/FS-SEI/FS-SEI_4800/Dataset/X_train_{num}Class.npy")
    y = np.load(f"D:/RFFI-based-on-CVCNN/codes/FS-SEI/FS-SEI_4800/Dataset/Y_train_{num}Class.npy")
    x=x[:,:,0]+x[:,:,1]*1j
    x=x.astype(np.complex64)
    x=np.expand_dims(x,axis=(1,3))
    # print(type(x[0,0,0,0]))
    y = y.astype(np.uint8)# 8bit数0-255
    X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size = 0.1, random_state= 30)
    return X_train, X_val, Y_train, Y_val

class FewShotDataset(Dataset):
    def __init__(self,datas,labels):
        self.datas=datas
        self.labels=labels

    def __getitem__(self, index):
        data=self.datas[index]
        label=self.labels[index]
        return data ,label

    def __len__(self):
        return len(self.labels)
X_train, X_val, Y_train, Y_val=TrainDataset(10)
trainDataset=FewShotDataset(X_train,Y_train)

train_loader=DataLoader( trainDataset,
                            batch_size=4,
                            num_workers=0,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True
                            )
for (data,label),i in zip(train_loader,range(5)):
        print(data[0,0,:10,0],label)

<class 'numpy.complex64'>
tensor([-5.1476e-05+4.1297e-06j, -4.2615e-05+5.8232e-06j, -3.3640e-05+8.2810e-06j,
        -2.6752e-05+1.0537e-05j, -2.3304e-05+1.2073e-05j, -2.3640e-05+1.2904e-05j,
        -2.7253e-05+1.3417e-05j, -3.3117e-05+1.4063e-05j, -4.0031e-05+1.5033e-05j,
        -4.6858e-05+1.6063e-05j]) tensor([5, 4, 8, 5], dtype=torch.uint8)
tensor([-6.3581e-05+6.4066e-05j, -7.1946e-05+5.1448e-05j,
        -8.1688e-05+3.4275e-05j, -9.1580e-05+1.4346e-05j,
        -9.9685e-05-5.7621e-06j, -1.0405e-04-2.3364e-05j,
        -1.0343e-04-3.6449e-05j, -9.7749e-05-4.4286e-05j,
        -8.8083e-05-4.7586e-05j, -7.6215e-05-4.8146e-05j]) tensor([9, 7, 8, 5], dtype=torch.uint8)
tensor([-1.1414e-06-6.2476e-05j,  3.3214e-06-6.0120e-05j,
         8.3174e-06-5.5408e-05j,  1.3659e-05-5.0050e-05j,
         1.9156e-05-4.5617e-05j,  2.4383e-05-4.3100e-05j,
         2.8593e-05-4.2759e-05j,  3.0876e-05-4.4285e-05j,
         3.0534e-05-4.7132e-05j,  2.7482e-05-5.0821e-05j]) tensor([6, 5, 7, 4], dtype=to

In [3]:
print(trainDataset[0][0].shape)

(1, 4800, 1)


In [3]:
for (data,label),i in zip(train_loader,range(5)):
    print(i)

In [4]:
file='WiFi_air_X330_3123D7B_2ft_run1.sigmf-data'
classes=['X310','X320']
file_class=None
for c in classes:
        if c in file:
                file_class=c
assert file_class!=None,"the  file's class is not matched with class list"
print(file_class)

AssertionError: the  file's class is not matched with class list